# Preparation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS598_DL4H'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
os.chdir(GOOGLE_DRIVE_PATH)

['mimic3-benchmarks-master', 'prepare_dataset.ipynb']


In [4]:
%cd mimic3-benchmarks-master

/content/drive/My Drive/CS598_DL4H/mimic3-benchmarks-master


In [5]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy


import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn.functional as F

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils


# Data Preprocessing

In [6]:
TIMESTEP = 1.0
data_path = 'data/hospital-mortality/'
# Build readers, discretizers, normalizers
train_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'train'),
                                         listfile=os.path.join(data_path, 'train_listfile.csv'),
                                         period_length=48.0)

val_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'train'),
                                       listfile=os.path.join(data_path, 'val_listfile.csv'),
                                       period_length=48.0)

discretizer = Discretizer(timestep=TIMESTEP,
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

In [7]:
discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
normalizer_state = 'ihm_normalizer'
normalizer_state = os.path.join(os.path.dirname(data_path), normalizer_state)
normalizer.load_params(normalizer_state)

In [8]:
train_raw = utils.load_data(train_reader, discretizer, normalizer, return_names=True)
val_raw = utils.load_data(val_reader, discretizer, normalizer, return_names=True)

In [9]:
demographic_data = []
with open('demographic_data.txt', 'r') as f:
  for line in f:
    demographic_data.append(np.fromstring(line, dtype=float, sep=' '))

diagnosis_data = []
with open('diagnosis_data.txt', 'r') as f:
  for line in f:
    diagnosis_data.append(np.fromstring(line, dtype=int, sep=' '))

idx_list = []
with open('idx_list.txt', 'r') as f:
  for line in f:
    idx_list.append(line.strip())

len(demographic_data)

<ipython-input-9-761523da05f3>:9: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  diagnosis_data.append(np.fromstring(line, dtype=int, sep=' '))


42125

In [10]:
class Dataset(Dataset):
  def __init__(self, x, y, name):
    self.x = x
    self.y = y
    self.name = name
  
  def __getitem__(self, index):
    return self.x[index], self.y[index], self.name[index]
  
  def __len__(self):
    return len(self.x)

In [185]:
BATCH_SIZE = 256
train_dataset = Dataset(train_raw['data'][0], train_raw['data'][1], train_raw['names'])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = Dataset(val_raw['data'][0], val_raw['data'][1], val_raw['names'])
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ConCare Model

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
print("available device: {}".format(device))

available device: cuda:0


In [12]:
class DemographicEmbed(nn.Module):
  def __init__(self, demo_input_dim, hidden_dim, dropout=0.5):
    super().__init__()
    self.demo_input_dim = demo_input_dim
    self.hidden_dim = hidden_dim

    self.fc = nn.Linear(self.demo_input_dim, self.hidden_dim)
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, x):
    return self.dropout(F.sigmoid(self.fc(x)))

In [13]:
class TimeAwareSelfAttention(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.w_qs = nn.Parameter(torch.randn(input_dim, hidden_dim))
    self.w_ks = nn.Parameter(torch.randn(input_dim, hidden_dim))
    self.beta = nn.Parameter(torch.zeros(1)+0.8)

    nn.init.kaiming_uniform_(self.w_qs, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.w_ks, a=math.sqrt(5))
  
  def forward(self, input):
    b, time_step, input_dim = input.size()
    time_decays = torch.tensor(range(47, -1, -1), dtype=torch.float32).unsqueeze(-1).unsqueeze(0).to(device)
    b_time_decays = time_decays.repeat(b, 1, 1)

    q = torch.matmul(input[:, -1, :], self.w_qs).reshape(b, 1, self.hidden_dim) # B*1*H
    k = torch.matmul(input, self.w_ks) # B*T*H
    v = input # B*T*H
    product = torch.matmul(q, k.transpose(1, 2)).squeeze() # B*T
    denominator = self.beta * torch.log(math.e + (1 - F.sigmoid(product)) * (b_time_decays.squeeze()))
    weights = F.softmax(F.tanh(product / denominator)) # B*T
    output = torch.matmul(weights.unsqueeze(1), v).squeeze() # B*H

    return output, weights

In [14]:
class ScaledDotProductAttention(nn.Module):
  def __init__(self):
    super().__init__()
    
  def forward(self, q, k, v):
    d_k = q.size(-1)
    attn = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    attn = F.softmax(attn, dim=-1)
    output = torch.matmul(attn, v)

    return output, attn

In [15]:
def cov(u):
  miu = torch.mean(u, dim=1).unsqueeze(1)
  diff = u - miu
  cov = torch.mm(diff, diff.T) / u.size(1)
  return cov

def cross_head_decorr(u):
  covs = cov(u[0, :, :])
  loss = 0.5 * (torch.norm(covs, p='fro') ** 2 - torch.norm(torch.diag(covs)) ** 2)
  for i in range(u.size(0)-1):
    covs = cov(u[i+1, :, :])
    loss += 0.5 * (torch.norm(covs, p='fro') ** 2 - torch.norm(torch.diag(covs)) ** 2)
  return loss

In [16]:
class MultiHeadAttention(nn.Module):
  def __init__(self, n_head, d_model, dropout=0.5):
    super().__init__()
    self.n_head = n_head
    self.d_model = d_model

    self.w_qs = nn.Linear(d_model, d_model, bias=False)
    self.w_ks = nn.Linear(d_model, d_model, bias=False)
    self.w_vs = nn.Linear(d_model, d_model, bias=False)
    self.fc = nn.Linear(d_model, d_model, bias=False)

    self.attn = ScaledDotProductAttention()

    self.dropout = nn.Dropout(dropout)
    self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

  def forward(self, q, k, v):
    b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)
    q = self.w_qs(q).view(b, len_q, self.n_head, self.d_model // self.n_head)
    k = self.w_ks(k).view(b, len_k, self.n_head, self.d_model // self.n_head)
    v = self.w_vs(v).view(b, len_v, self.n_head, self.d_model // self.n_head)
    q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

    q, attn = self.attn(q, k, v)

    q = q.transpose(1, 2).contiguous().view(b, len_q, -1)
    output_q = self.dropout(self.fc(q))

    # DeCov
    u = q.transpose(0, 1).transpose(1, 2)
    decov_loss = cross_head_decorr(u)

    return output_q, decov_loss


In [17]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, ff_hidden, dropout=0.5):
    super().__init__()
    self.fc1 = nn.Linear(d_model, ff_hidden)
    self.fc2 = nn.Linear(ff_hidden, d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    output = self.fc2(self.dropout(F.relu(self.fc1(x))))
    return output

In [18]:
class ResConnect(nn.Module):
  def __init__(self, norm_shape, dropout=0.5):
    super().__init__()
    self.lnorm = nn.LayerNorm(norm_shape)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, layer):
    res = layer(self.lnorm(x))
    output = x + self.dropout(res[0])
    return output, res[1]

In [19]:
class FinalAttention(nn.Module):
  def __init__(self, input_dim, hidden_dim, dropout=0.5):
    super().__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.w_qs = nn.Linear(input_dim, hidden_dim)
    self.w_ks = nn.Linear(input_dim, hidden_dim)
    self.w_vs = nn.Linear(input_dim, hidden_dim)

    # Parameter Initialization
    nn.init.kaiming_uniform_(self.w_qs.weight, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.w_ks.weight, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.w_vs.weight, a=math.sqrt(5))
    
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, input):
    batch_size, time_step, input_dim = input.size()
    q = self.w_qs(input[:, -1, :]).reshape(batch_size, self.hidden_dim, 1)
    k = self.w_ks(input)
    v = self.w_vs(input)

    attention = F.tanh(torch.matmul(k, q).squeeze())
    score = F.softmax(attention)

    if self.dropout is not None:
      score = self.dropout(score)

    v = torch.matmul(score.unsqueeze(1), v).squeeze()

    return v, score

In [20]:
class ConCare(nn.Module):
  def __init__(self, demo_input_dim, input_dim, hidden_dim1, hidden_dim2, d_model, n_head, ff_hidden, output_dim, dropout=0.5):
    super().__init__()
    self.demo_input_dim = demo_input_dim
    self.input_dim = input_dim
    self.hidden_dim1 = hidden_dim1
    self.hidden_dim2 = hidden_dim2
    self.d_model = d_model
    self.n_head = n_head
    self.ff_hidden = ff_hidden
    self.output_dim = output_dim
    self.dropout = dropout

    self.demo_embed = DemographicEmbed(self.demo_input_dim, self.hidden_dim1)
    self.gru = nn.GRU(1, self.hidden_dim1, batch_first=True)
    self.time_attn = TimeAwareSelfAttention(self.hidden_dim1, self.hidden_dim2)
    self.multi_attn = MultiHeadAttention(self.n_head, self.d_model)
    self.final_attn = FinalAttention(self.hidden_dim1, self.hidden_dim1, dropout=dropout)
    self.res = ResConnect(self.d_model, dropout=self.dropout)
    self.pos_ff = PositionwiseFeedForward(self.d_model, self.ff_hidden, dropout=self.dropout)
    self.output_fc = nn.Linear(self.hidden_dim1, self.output_dim)
  
  def forward(self, input, demo):
    # Demographic embedding
    demo_embedding = self.demo_embed(demo)

    # First record embedding
    batch_size = input.size(0)
    feat_dim = input.size(2)
    record_embedding1 = self.gru(input[:, :, 0].unsqueeze(-1), torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0).to(device))[0]
    time_attn_output = self.time_attn(record_embedding1)[0].unsqueeze(1)

    # All other records embeddings
    for i in range(1, feat_dim):
      embedding = self.gru(input[:, :, i].unsqueeze(-1), torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0).to(device))[0]
      attn = self.time_attn(embedding)[0].unsqueeze(1)
      time_attn_output = torch.cat((time_attn_output, attn), 1)
    
    # Combine with demographic embedding
    demo_embedding = demo_embedding.unsqueeze(1)
    time_attn_output = torch.cat((time_attn_output, demo_embedding), 1)

    # Get multi-head attention
    multi_attn_output, dev_loss = self.res(time_attn_output, lambda x: self.multi_attn(time_attn_output, time_attn_output, time_attn_output))
    final_input = self.res(multi_attn_output, lambda x: self.pos_ff(multi_attn_output))[0]

    # Get final output
    value, score = self.final_attn(final_input)
    output = F.sigmoid(self.output_fc(value))

    return output, dev_loss

# Training

In [21]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)

In [22]:
model = ConCare(demo_input_dim=12, input_dim=76, hidden_dim1=64, hidden_dim2=8, d_model=64, n_head=4, ff_hidden=256, output_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.BCELoss()

In [198]:
max_roc = 0
max_prc = 0
train_total_loss = []
val_total_loss = []
result_history = []
file_name = 'model/concare0'


for epoch in range(100):
  
  # Start training
  train_batch_total_loss = []
  model.train()

  for step, (batch_x, batch_y, batch_name) in enumerate(train_loader):
    optimizer.zero_grad()
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)

    # Get batch demographic data
    batch_demo = []
    for i in range(len(batch_name)):
      cur_id, cur_ep, _ = batch_name[i].split('_', 2)
      cur_idx = cur_id + '_' + cur_ep
      cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
      batch_demo.append(cur_demo)
    batch_demo = torch.stack(batch_demo).to(device)

    # Get model outputs
    output, decov_loss = model(batch_x, batch_demo)

    # Get loss
    loss = loss_func(output, batch_y.unsqueeze(-1))
    loss += decov_loss
    train_batch_total_loss.append(loss.cpu().detach().numpy())
    loss.backward()

    optimizer.step()

    if step % 50 == 0:
      print('Epoch %d Batch %d: Train Loss = %.4f'%(epoch, step, np.mean(np.array(train_batch_total_loss))))
    
  train_total_loss.append(np.mean(np.array(train_batch_total_loss)))
  print('Epoch %d: Train Loss = %.4f'%(epoch, np.mean(np.array(train_batch_total_loss))))

  # Start Validating
  val_batch_total_loss = []
  y_true = []
  y_pred = []

  with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(val_loader):
      batch_x = batch_x.float().to(device)
      batch_y = batch_y.float().to(device)

      # Get batch demographic data
      batch_demo = []
      for i in range(len(batch_name)):
        cur_id, cur_ep, _ = batch_name[i].split('_', 2)
        cur_idx = cur_id + '_' + cur_ep
        cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
        batch_demo.append(cur_demo)
      batch_demo = torch.stack(batch_demo).to(device)

      # Get model outputs
      output, decov_loss = model(batch_x, batch_demo)

      # Get loss
      loss = loss_func(output, batch_y.unsqueeze(-1))
      loss += decov_loss
      val_batch_total_loss.append(loss.cpu().detach().numpy())
      y_pred += list(output.cpu().detach().numpy())
      y_true += list(batch_y.cpu().numpy().flatten())
    
    val_total_loss.append(np.mean(np.array(val_batch_total_loss)))
    print('Epoch %d: Validation Loss = %.4f'%(epoch, np.mean(np.array(val_batch_total_loss))))

    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    result = metrics.print_metrics_binary(y_true, y_pred)
    result_history.append(result)

    cur_auroc = result['auroc']
    print('Epoch %d: Validation AUROC = %.4f'%(epoch, cur_auroc))
    
    if cur_auroc > max_roc:
        max_roc = cur_auroc
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch
        }
        torch.save(state, file_name)
        print('\n------------ Save best model ------------\n')

<ipython-input-188-afdceca3a47c>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-194-a3370d8276f2>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 0 Batch 0: Train Loss = 0.6550
Epoch 0 Batch 50: Train Loss = 0.4276
Epoch 0: Train Loss = 0.4236
Epoch 0: Validation Loss = 0.3986
confusion matrix:
[[2726    0]
 [ 432    0]]
accuracy = 0.8632045388221741
precision class 0 = 0.8632045388221741
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.4722230713839298
AUC of PRC = 0.12724271411033997
min(+P, Se) = 0.14481409001956946
f1_score = nan
Epoch 0: Validation AUROC = 0.4722

------------ Save best model ------------



/content/drive/My Drive/CS598_DL4H/mimic3-benchmarks-master/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
<ipython-input-188-afdceca3a47c>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-194-a3370d8276f2>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 1 Batch 0: Train Loss = 0.3433
Epoch 1 Batch 50: Train Loss = 0.4014
Epoch 1: Train Loss = 0.4014
Epoch 1: Validation Loss = 0.4006
confusion matrix:
[[2726    0]
 [ 432    0]]
accuracy = 0.8632045388221741
precision class 0 = 0.8632045388221741
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.6003055283823809
AUC of PRC = 0.18205209712450593
min(+P, Se) = 0.21484375
f1_score = nan
Epoch 1: Validation AUROC = 0.6003

------------ Save best model ------------



/content/drive/My Drive/CS598_DL4H/mimic3-benchmarks-master/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
<ipython-input-188-afdceca3a47c>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-194-a3370d8276f2>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


...

Epoch 97 Batch 0: Train Loss = 0.3129
Epoch 97 Batch 50: Train Loss = 0.3499
Epoch 97: Train Loss = 0.3502
Epoch 97: Validation Loss = 0.3525
confusion matrix:
[[2718    8]
 [ 418   14]]
accuracy = 0.8651044964790344
precision class 0 = 0.8667091727256775
precision class 1 = 0.6363636255264282
recall class 0 = 0.9970653057098389
recall class 1 = 0.032407406717538834
AUC of ROC = 0.7517042675470775
AUC of PRC = 0.333527572141141
min(+P, Se) = 0.35185185185185186
f1_score = 0.06167400785396493
Epoch 97: Validation AUROC = 0.7517


<ipython-input-188-afdceca3a47c>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-194-a3370d8276f2>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 98 Batch 0: Train Loss = 0.3801
Epoch 98 Batch 50: Train Loss = 0.3521
Epoch 98: Train Loss = 0.3525
Epoch 98: Validation Loss = 0.3528
confusion matrix:
[[2712   14]
 [ 413   19]]
accuracy = 0.8647878170013428
precision class 0 = 0.8678399920463562
precision class 1 = 0.5757575631141663
recall class 0 = 0.994864284992218
recall class 1 = 0.04398148134350777
AUC of ROC = 0.7522273511589359
AUC of PRC = 0.33424595069177043
min(+P, Se) = 0.3534136546184739
f1_score = 0.08172042826832147
Epoch 98: Validation AUROC = 0.7522


<ipython-input-188-afdceca3a47c>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-194-a3370d8276f2>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 99 Batch 0: Train Loss = 0.3837
Epoch 99 Batch 50: Train Loss = 0.3459
Epoch 99: Train Loss = 0.3488
Epoch 99: Validation Loss = 0.3515
confusion matrix:
[[2721    5]
 [ 425    7]]
accuracy = 0.8638378977775574
precision class 0 = 0.8649078011512756
precision class 1 = 0.5833333134651184
recall class 0 = 0.9981657862663269
recall class 1 = 0.016203703358769417
AUC of ROC = 0.7532038871226326
AUC of PRC = 0.32723007249111546
min(+P, Se) = 0.3618421052631579
f1_score = 0.03153153094738569
Epoch 99: Validation AUROC = 0.7532


# Test

In [23]:
file_name = 'model/concare0'
BATCH_SIZE = 256

checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'test'),
                                            listfile=os.path.join(data_path, 'test_listfile.csv'),
                                            period_length=48.0)
test_raw = utils.load_data(test_reader, discretizer, normalizer, return_names=True)
test_dataset = Dataset(test_raw['data'][0], test_raw['data'][1], test_raw['names'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [25]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(test_loader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_demo = []
        for i in range(len(batch_name)):
            cur_id, cur_ep, _ = batch_name[i].split('_', 2)
            cur_idx = cur_id + '_' + cur_ep
            cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
            batch_demo.append(cur_demo)

        batch_demo = torch.stack(batch_demo).to(device)
        output = model(batch_x, batch_demo)[0]

        loss = loss_func(output, batch_y.unsqueeze(-1))
        batch_loss.append(loss.cpu().detach().numpy())
        y_pred += list(output.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\nTest Prediction Result")
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

<ipython-input-13-afdceca3a47c>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-19-a3370d8276f2>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)



Test Prediction Result
confusion matrix:
[[2861    1]
 [ 373    1]]
accuracy = 0.8844252228736877
precision class 0 = 0.8846629858016968
precision class 1 = 0.5
recall class 0 = 0.9996505975723267
recall class 1 = 0.002673796843737364
AUC of ROC = 0.7537481735594943
AUC of PRC = 0.2976196014396501
min(+P, Se) = 0.32981530343007914
f1_score = 0.005319148955880186
